# 01. Transforming Video file(.avi) to 160*160 size, 30 frames Numpy array(.npy)
* **Purpose of this Code** : Making train set & test set to train and evaluate Model
* **`Please!`** **Before Run this code, You should separate your videos to Fight folder / NonFight Folder**
* **`Output=.pickle`** : Very Large file.(Mabye about 30~50GB at each one?)
    * Please check your drive capacity, before run this code!

# Imports

In [1]:
import os
import pickle # save list as .pickle
import numpy as np
from tqdm import tqdm
import cv2 # read video file
from skimage.transform import resize # resizing images

# 01-A. Transform video files to Numpy array

In [2]:
def Save2Npy(file_dir, save_dir):
    """This function load videos, and transform each of them to Numpy array, and save them in selected folder.
    :: file_dir :: This folder have original video files
    :: save_dir :: You'll save transformed Numpy array in this folder.
    """
    if not os.path.exists(save_dir): # If there is not save_dir folder, then create new folder at there.
        os.makedirs(save_dir)
    
    file_list=os.listdir(file_dir) # Make list of video file(in file_dir folder)'s name.
    
    for file in tqdm(file_list):
        frames=np.zeros((30, 160, 160, 3), dtype=np.float)
        i=0
        
        vid=cv2.VideoCapture(os.path.join(file_dir, file)) # Create cv2.VideoCapture() Object of each video file.
        
        if vid.isOpened():
            grabbed, frame=vid.read()
        else:
            grabbed=False
            
        frm=resize(frame, (160, 160, 3))
        frm=np.expand_dims(frm, axis=0)
        
        if(np.max(frm)>1):
            frm=frm/255.0
        frames[i][:]=frm
        i+=1
      
        while i<30:
            grabbed, frame=vid.read()
            frm=resize(frame, (160, 160, 3))
            frm=np.expand_dims(frm, axis=0)
            if(np.max(frm)>1):
                frm=frm/255.0
            frames[i][:] = frm
            i+=1

        video_name=file.split('.')[0]
        video_path=os.path.join(file_dir, file)
        save_path=os.path.join(save_dir, video_name+'.npy')

        np.save(save_path, frames)

In [3]:
file_dir='D:/datasets/itwill/Fight_210511' # Folder that have videos
save_dir='D:/datasets/AllVideo_Fight_Numpy'

In [4]:
Save2Npy(file_dir=file_dir, save_dir=save_dir)

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:08<00:00,  1.89it/s]


In [5]:
file_dir='D:/datasets/itwill/NonFight'
save_dir='D:/datasets/AllVideo_NonFight_Numpy'

In [6]:
Save2Npy(file_dir=file_dir, save_dir=save_dir)

100%|██████████████████████████████████████████████████████████████████████████████████| 55/55 [02:56<00:00,  3.21s/it]


# 01-B. Make list of Numpy arrays

## 1. Fight Videos

In [5]:
Fight_dir='D:/datasets/AllVideo_Fight_Numpy' # 영상 넘파이 배열들이 있는 폴더
file_list_npy = os.listdir(Fight_dir) # 넘파이 배열들이 저장된 폴더(save_dir) 내의 파일이름들

data_Fight=[]
for file in file_list_npy: # 해당 폴더 내의 전체 넘파이들에 대해서
    file_path=os.path.join(Fight_dir, file)
    x=np.load(file_path)
    data_Fight.append(x)

print(len(data_Fight))

1857


## 2. NonFight Videos

In [8]:
NonFight_dir='D:/datasets/AllVideo_NonFight_Numpy'
file_list_npy=os.listdir(NonFight_dir)

data_NonFight=[]
for file in file_list_npy:
    file_path=os.path.join(NonFight_dir, file)
    x=np.load(file_path)
    data_NonFight.append(x)

print(len(data_NonFight))

1741


# 01-C. Save list as .pickle

In [6]:
with open("D:/datasets/AllVideo_numpy_list_pickle/01_data_Fight_210512.pickle","wb") as fw:
    pickle.dump(data_Fight, fw, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
with open("D:/datasets/AllVideo_numpy_list_pickle/01_data_NonFight_210507.pickle","wb") as fw:
    pickle.dump(data_NonFight, fw, protocol=pickle.HIGHEST_PROTOCOL)

# 01-D. Make & Save Label(Real Values) as .pickle
> Memo
```
prediction=preds.argmax(axis=0):[0 0]
Results =  [[0.09166703 0.9092251 ]]
Maximun Probability =  0.9092251
Difference of prob  0.8184502124786377
Alert : violence - 90.92%
```
> labels1=[] : list of each video file's lavel(Fight/NonFight). 
> * Violence(Fight) : [0,1]
> * Non-Violence(NonFight) : [1,0]

## 1. Make Label list

In [7]:
label_Fight_per_video=np.array([0,1])
label_Fight=[label_Fight_per_video]*len(data_Fight) # As amount as count of Violence(Fight) Video

In [12]:
label_NonFight_per_video=np.array([1,0])
label_NonFight=[label_NonFight_per_video]*len(data_NonFight) # As amount as count of Non-Violence(NonFight) Video

In [13]:
len(label_Fight), len(label_NonFight)

(1841, 1741)

In [14]:
label_Fight[0], label_NonFight[0]

(array([0, 1]), array([1, 0]))

## 2. Save Label list as .pickle

In [8]:
with open("D:/datasets/AllVideo_numpy_list_pickle/01_label_Fight_210512.pickle","wb") as fw:
    pickle.dump(label_Fight, fw)

In [16]:
with open("D:/datasets/AllVideo_numpy_list_pickle/01_label_NonFight_210507.pickle","wb") as fw:
    pickle.dump(label_NonFight, fw)